In [3]:
import pandas as pd
import csv

csv.field_size_limit(500 * 1024 * 1024)
 
df=pd.read_csv('./youtube_all_results.csv', engine='python')

In [140]:
df[~df['isKnownLink'].isna()].shape

(3828, 8)

In [141]:
df[(~df['isUTMlink'].isna()) & df['isKnownLink'].isna()].shape

(7558, 8)

In [142]:
df_isAffiliate = df[~df['isKnownLink'].isna()]
df_maybeAffiliate = df[(~df['isUTMlink'].isna()) & df['isKnownLink'].isna()]

In [143]:
affiliate_url_list=df_isAffiliate['url'].tolist()
affiliate_url_sus_list=df_maybeAffiliate['url'].tolist()

In [144]:
df_youtube_urls=pd.read_feather('./all-URL-no-filters.feather')
urls = df_youtube_urls.iloc[:, -1].values
result = []
for url_list in urls:
    for url in url_list:
        if url.startswith('http://') or url.startswith('https://'):
            result.append(url)
output = pd.DataFrame(data=result, columns=['url'])
output.to_csv('./youtube-all-urls.csv', index = False)

In [146]:
output['isAffiliate']=0

In [147]:
output

,url,isAffiliate
0,https://mhzacademy.spayee.com/s/store,0
1,https://forms.gle/T2eexAjR3ypK9DYW8,0
2,https://mhzacademy.com/cpa-marketing-roadmap/,0
3,https://youtu.be/Sew4rctKghY,0
4,http://originalroyalty.com/product/aviyah-drea...,0
...,...,...
445844,https://bit.ly/3ya1bDl,0
445845,https://bit.ly/3ncX5oL,0
445846,https://bit.ly/3NcsBxz,0
445847,https://bit.ly/3Ni8IVO,0


In [148]:
from tqdm import tqdm

for i in tqdm(range(len(output))):
    if output.loc[i, 'url'] in affiliate_url_list:
        output.loc[i, 'isAffiliate']=1
    elif output.loc[i, 'url'] in affiliate_url_sus_list:
        output.loc[i, 'isAffiliate']=2

100%|██████████| 445849/445849 [01:22<00:00, 5371.96it/s]


In [149]:
output

,url,isAffiliate
0,https://mhzacademy.spayee.com/s/store,0
1,https://forms.gle/T2eexAjR3ypK9DYW8,0
2,https://mhzacademy.com/cpa-marketing-roadmap/,0
3,https://youtu.be/Sew4rctKghY,0
4,http://originalroyalty.com/product/aviyah-drea...,0
...,...,...
445844,https://bit.ly/3ya1bDl,1
445845,https://bit.ly/3ncX5oL,0
445846,https://bit.ly/3NcsBxz,0
445847,https://bit.ly/3Ni8IVO,2


In [150]:
output[output['isAffiliate']==1].shape

(8944, 2)

In [151]:
output[output['isAffiliate']==2].shape

(21643, 2)

In [157]:
df_youtube_all_affiliate_url = output[output['isAffiliate'] == 1]
df_youtube_all_sus_affiliate_url = output[output['isAffiliate'] == 2]

In [159]:
df_youtube_all_affiliate_url = df_youtube_all_affiliate_url.reindex(
    columns=df_youtube_all_affiliate_url.columns.tolist() + ["landing_page"])
df_youtube_all_sus_affiliate_url = df_youtube_all_sus_affiliate_url.reindex(
    columns=df_youtube_all_sus_affiliate_url.columns.tolist() + ["landing_page"])


In [160]:
df_youtube_all_affiliate_url.reset_index(drop=True)
df_youtube_all_sus_affiliate_url.reset_index(drop=True)

,url,isAffiliate,landing_page
0,http://bit.ly/CareersCloud_App,2,NaN
1,http://bit.ly/CareersCloud_App,2,NaN
2,http://bit.ly/CareersCloud_App,2,NaN
3,http://bit.ly/CareersCloud_App,2,NaN
4,http://bit.ly/CareersCloud_App,2,NaN
...,...,...,...
21638,https://bit.ly/3y2B7ty,2,NaN
21639,https://bit.ly/3ueYE9I,2,NaN
21640,https://bit.ly/3Nl4ZqC,2,NaN
21641,https://bit.ly/3Ni8IVO,2,NaN


In [161]:
df_youtube_all_affiliate_url['landing_page']=None
df_youtube_all_sus_affiliate_url['landing_page']=None

In [163]:
# test
a = df_youtube_all_affiliate_url.iloc[0, 0]
df[df['url']==a].landing_page.tolist()[0]

'https://www.amazon.in/gp/product/B004XIH3DC/ref=as_li_tl?ie=UTF8&camp=3638&creative=24630&creativeASIN=B004XIH3DC&linkCode=as2&tag=etvwp-21&linkId=d8f7179486e6ef1d5a35f19651277c45'

In [164]:
for i in tqdm(range(len(df_youtube_all_affiliate_url))):
    current_url = df_youtube_all_affiliate_url.iloc[i,0]
    current_landing_page = df[df['url']==current_url].landing_page.tolist()
    if current_landing_page:
        df_youtube_all_affiliate_url.iloc[i, 2] = current_landing_page[0]


100%|██████████| 8944/8944 [00:32<00:00, 271.99it/s]


In [166]:
for i in tqdm(range(len(df_youtube_all_sus_affiliate_url))):
    current_url = df_youtube_all_sus_affiliate_url.iloc[i, 0]
    current_landing_page = df[df['url'] == current_url].landing_page.tolist()
    if current_landing_page:
        df_youtube_all_sus_affiliate_url.iloc[i, 2] = current_landing_page[0]


100%|██████████| 21643/21643 [01:25<00:00, 252.76it/s]


In [172]:
df_youtube_all_affiliate_url.reset_index(drop=True)

,url,isAffiliate,landing_page
0,https://amzn.to/3syiwD1,1,https://www.amazon.in/gp/product/B004XIH3DC/re...
1,https://amzn.to/3syiwD1,1,https://www.amazon.in/gp/product/B004XIH3DC/re...
2,https://amzn.to/35SC2Cm,1,https://www.amazon.in/gp/product/B07K48339S/re...
3,https://amzn.to/3syiwD1,1,https://www.amazon.in/gp/product/B004XIH3DC/re...
4,https://amzn.to/3HGwdpT,1,https://www.amazon.in/gp/product/B00Q70JLUM/re...
...,...,...,...
8939,https://bit.ly/3uaZVyg,1,https://www.amazon.com/Nana-Loves-More-Jimmy-F...
8940,https://bit.ly/3tTQ1AX,1,http://pepperjamnetwork.com/tracking/error.php
8941,https://bit.ly/3ygbBCq,1,http://pepperjamnetwork.com/tracking/error.php
8942,https://bit.ly/3tYT2A2,1,https://www.amazon.com/Echo-Frames/dp/B08T5P7B...
